In [1]:
import pandas as pd

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [9]:
path = "../../../Data/ai_human_images/"
test = pd.read_csv(path+"test.csv")
pred = pd.read_csv(path+"predictions.csv", index_col=0)

In [11]:
test["label"] = pred
test.to_csv("submission.csv", index=False)